In [61]:
# Import all cars from opendbc

from opendbc.car import structs
from opendbc.car.values import PLATFORMS as TEST_PLATFORMS

# Apply any platform selection/filtering here

print(f"Searching {len(TEST_PLATFORMS)} platforms")

Searching 221 platforms


In [62]:
import random

from openpilot.tools.lib.logreader import LogReader
from openpilot.tools.lib.comma_car_segments import get_comma_car_segments_database

database = get_comma_car_segments_database()
TEST_SEGMENTS = []

MAX_SEGS_PER_PLATFORM = 3  # Increase this to search more segments

for platform in TEST_PLATFORMS:
  if platform not in database:
    print(f"No segments available for {platform}")
    continue
  
  all_segments = database[platform]
  NUM_SEGMENTS = min(len(all_segments), MAX_SEGS_PER_PLATFORM)
  TEST_SEGMENTS.extend(random.sample(all_segments, NUM_SEGMENTS))

print(f"Searching {len(TEST_SEGMENTS)} segments")


No segments available for DODGE_DURANGO
No segments available for FORD_RANGER_MK2
No segments available for HOLDEN_ASTRA
No segments available for CADILLAC_ATS
No segments available for CHEVROLET_MALIBU
No segments available for CADILLAC_XT4
No segments available for CHEVROLET_VOLT_2019
No segments available for CHEVROLET_TRAVERSE
No segments available for GMC_YUKON
No segments available for HONDA_ODYSSEY_CHN
No segments available for HYUNDAI_KONA_2022
No segments available for HYUNDAI_NEXO_1ST_GEN
No segments available for GENESIS_GV70_ELECTRIFIED_1ST_GEN
No segments available for GENESIS_G80_2ND_GEN_FL
No segments available for RIVIAN_R1_GEN1
No segments available for SUBARU_FORESTER_HYBRID
No segments available for TESLA_MODEL_3
No segments available for TESLA_MODEL_Y
No segments available for TOYOTA_RAV4_PRIME
No segments available for TOYOTA_SIENNA_4TH_GEN
No segments available for LEXUS_LC_TSS2
No segments available for VOLKSWAGEN_CADDY_MK3
No segments available for VOLKSWAGEN_CR

In [ ]:
from openpilot.tools.lib.logreader import LogReader
from tqdm.notebook import tqdm, tnrange

# Example search for CAN ignition messages
# Be careful when filtering by bus, account for external Red Panda or odd harness arrangements on Honda/HKG

BUSES_TO_SEARCH = [0, 1, 2]

MESSAGES_TO_FIND = {
  0x1F1: "GM CAN Ign",
  0x152: "Rivian CAN Ign",
  0x221: "Tesla 3/Y CAN Ign",
  0x9E:  "Mazda CAN Ign",
  0x3C0: "VW CAN Ign",
}

progress_bar = tnrange(len(TEST_SEGMENTS), desc="segments searched")

for segment in TEST_SEGMENTS:
  lr = LogReader(segment)
  CP = lr.first("carParams")
  if CP is None:
    continue

  can_packets = [msg for msg in lr if msg.which() == "can"]
  matched_messages = set()

  for packet in can_packets:
    for msg in packet.can:
      if msg.address in MESSAGES_TO_FIND and msg.src in BUSES_TO_SEARCH:
        # print(msg)
        matched_messages.add(msg.address)

  if len(matched_messages) > 0:
    message_names = [MESSAGES_TO_FIND[message] for message in matched_messages]
    print(f"Match found: {segment:<45} {CP.carFingerprint:<38} {message_names}")

  progress_bar.update()


segments searched:   0%|          | 0/577 [00:00<?, ?it/s]

Match found: 0f53b336851e1384/2024-01-14--09-23-09/22/s    CHRYSLER PACIFICA HYBRID 2018          ['VW CAN Ign']
Match found: 268be6212ca01164/2023-12-13--14-24-23/9/s     CHRYSLER PACIFICA HYBRID 2018          ['VW CAN Ign']
Match found: d753e5427226080e/2023-12-22--20-03-07/14/s    CHRYSLER PACIFICA HYBRID 2018          ['VW CAN Ign']
Match found: ec8aef9196cb9d87/2023-12-09--09-32-50/4/s     CHRYSLER PACIFICA HYBRID 2019          ['VW CAN Ign']
Match found: 35176353a674cbdc/2023-06-04--18-26-00/52/s    CHRYSLER PACIFICA HYBRID 2019          ['VW CAN Ign']
Match found: 10159b993b12b295/2024-01-16--22-13-36/15/s    CHRYSLER PACIFICA HYBRID 2019          ['VW CAN Ign']
Match found: 12e966025220f57c/2023-12-31--11-02-21/450/s   CHRYSLER PACIFICA 2018                 ['VW CAN Ign']
Match found: 7e705eb5c27a49cc/2024-01-16--14-16-00/4/s     CHRYSLER PACIFICA 2018                 ['VW CAN Ign']
Match found: 7e705eb5c27a49cc/2023-11-22--14-04-22/12/s    CHRYSLER PACIFICA 2018               